In [1]:
# Data Analysis Libraries
import pandas as pd
import numpy as np
import math

# Web Scrapping Libraries
import requests
from bs4 import BeautifulSoup

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns',None)

In [2]:
url_resenhas = 'https://www.skoob.com.br/livro/resenhas/{}/mais-gostaram/mpage:{}' 
# página do Skoob em que constam as resenhas, em que o primeiro {} será o número do livro e o segundo {} a página de reviews


In [34]:
def raspar_skoob(livro_inicial, livro_final, limite,imprimir):
    # livro_inicial: primeiro livro a ser coletado (número inteiro não nulo)
    # livro_final: ultimo livro a ser coletado, incluindo-o (sendo necessáriamente um número maior que o do livro_inicial)
    # limite: limite de reviews a serem coletados. 
        # Caso o valor seja menor que o número de reviews existentes, ele limitárá o total
        # Caso insira um valor maior que o número de reviews disponíveis, ele pegará o máximo existente
        # Caso não deseje limitar, inserir o valor 0
    # imprimir: se True, irá printar os estágios, se False, não irá printar os estágios
    
    lista_completa = [] # lista com output de todas as reviews coletadas
    lista_livros = [] # lista dos livros coletados
    
    
    # range da lista de livros, começando no livro_inicial até livro_final, em que será feita a coleta
    for i in range(livro_inicial,livro_final+1): 
        response = requests.get(url_resenhas.format(i,'1'))
        pagina_html = BeautifulSoup(response.text)
        corpo = pagina_html.findAll(id="corpo") # corpo da página a ser feita a raspagem de dados (Web Scraping)
        
        try: # testar se existe um livro naquela posição, caso não exista simplesmente passará para o próximo livro
            
            Livro = corpo[0].findAll('strong')[0].get_text() # Nome do Livro
            Autor = corpo[0].findAll('a')[1].get_text() # Nome do Autor 
            
            # Salvar o livro, após verificar que existem livros
            lista_livros.append({'author':Autor,
                                 'book':Livro,
                                 'link':'https://www.skoob.com.br/livro/resenhas/{}/mais-gostaram/mpage:{}'.format(i,'1'),
                                 'livro_numero':i})
            
            try: # testar se existe alguma reviews para este livro, se não houver, anotará o livro mas com reviews vazias (None)

                # calculando número de páginas com comentários
                total_pags = math.ceil(int(corpo[0].findAll(id="perfil-conteudo-intern")[0].findAll('b')[0].get_text().split(' ')[0])/15)
                
                if limite == 0: # se limite for definido como 0, pegará todas os comentários existentes   
                    pass            
                elif total_pags > math.ceil(limite/15): # se houverem mais páginas que as necessárias para encontrar o limite
                    total_pags = math.ceil(limite/15) # limitará as páginas de modo a limitar o total de comentários
                else:
                    pass # se não for 0 nem maior que o existente, teremos menos reviews que o desejado, portanto pegaremos todos os disponíveis
                
                if imprimir: # se True, irá printar o livro que está sendo coletado
                    # printando nome do livro e indice do site, para identificar casos de bug
                    print('Livro {}: {}'.format(i, Livro)) 
                
                contagem  = 0 # definindo um contador de comentários por livro
                
                for paginas in range(1,total_pags+1): # range das páginas com comentários, seguindo limite caso tenha sido definido
                    response2 = requests.get(url_resenhas.format(i,paginas))
                    pagina_html2 = BeautifulSoup(response2.text)
                    corpo2 = pagina_html2.findAll(id="corpo") # corpo das reviews de cada página
                    
                    if imprimir: # se True, irá printar o livro e página que está sendo coletado
                        print('Livro {}: {} - Página: {}'.format(i, Livro, paginas)) # printando página da review, para identificar casos de bug
                    
                    # existem até 15 reviews por página, cada uma com o ID da review aparecendo 2x, portanto
                    # o range padrão de coleta de comentários por página vai de 0 a 29, pulando de 2 em 2
                    # no entanto, possívelmente ele foi limitado
                    
                    if (paginas == math.ceil(limite/15) and limite!= 0): # se a página atual for a mesma definida ao limitar
                        ultima_review = limite-contagem # encontrar quantas reviews ainda serão coletadas para atingir limite definido
                        ultima_review = ultima_review*2
                    else:
                        ultima_review = 30 # se nao estiver no limite definido, usará o range completo das reviews
                    
                    for resenhas in range(0,ultima_review,2):                       
                            
                        try: # testar se existe comentário nesta posição, uma vez que se nao tiver sido limitado, 
                             # na última página de comentários pode haver menos de 15
                            if imprimir: # se True, irá printar o livro e página e review que está sendo coletado
                                print('Livro {}: {} - Página: {} - Review: {}'.format(i, Livro, paginas,int(resenhas/2+1))) # printando a posição da review, para identificar casos de bug
                            
                            # dados gerais da resenha    
                            review_pt1 = corpo2[0].findAll(id="perfil-conteudo-intern")[0].findAll("div", id=lambda value: value and value.startswith("resenha"))[resenhas]
                            
                            Link = 'https://www.skoob.com.br/livro/resenhas/{}/mais-gostaram/mpage:{}'.format(i,paginas) # link da review
                            id_resenha = review_pt1.get('id') # ID da review
                            id_usuario = review_pt1.find('a').get('href').split('/')[-1] # ID do usuário
                            nota = float(review_pt1.find('star-rating').get('rate')) # Nota dada pelo usuário
                            
                            # dados de texto da resenha
                            review_pt2 = corpo2[0].findAll(id="perfil-conteudo-intern")[0].findAll("div", id=lambda value: value and value.startswith("resenha"))[resenhas+1]
                            
                            try: # caso exista um título, salvar ele separadamente
                                titulo = review_pt2.findAll('strong')[1].get_text() 
                            except IndexError: # caso não exista um título, salvar None
                                titulo = None
                                
                            resenha = review_pt2.get_text() # obtendo o texto que consta título, data e resenha
                            
                            # na estrutura obtida, caso exista um titulo, ele é a ultima informação antes da resenha em si
                            # caso não exista um título, a data é a ultima informação
                            if titulo == None: # caso não haja um título, usar a data como divisor para obter apenas a resenha
                                resenha = resenha.split('/',2)[-1][4:]
                            else: # caso haja um título, utilizá-lo como divisor para obter a resenha
                                resenha = resenha.split(titulo,1)[1]
                
                            # salvar as resenhas na lista de resenhas
                            lista_completa.append({'author':Autor,
                                                   'book':Livro,
                                                   'link':Link,
                                                   'review_id': id_resenha,
                                                   'user_id': id_usuario,
                                                   'livro_pagina_posicao':'{}-{}-{}'.format(i,paginas,int(resenhas/2+1)),
                                                   'rating': nota,
                                                   'review_title': titulo,
                                                   'review': resenha})
                            contagem+=1
                            
                        except IndexError: # caso não exista um comentário nessa posição, irá quebrar o loop, mudanodo de livro
                            break
            except IndexError: # no caso do livro não ter comentários, salvará o livro sem comentários
                lista_completa.append({'author':Autor,
                                       'book':Livro,
                                       'link':'https://www.skoob.com.br/livro/resenhas/{}/mais-gostaram/mpage:{}'.format(i,'1'),
                                       'review_id': None,
                                       'user_id': None,
                                       'livro_pagina_posicao':'{}-{}-{}'.format(1,1,0),
                                       'rating': None,
                                       'review_title': None,
                                       'review': None})
                
        except IndexError: # caso não exista um livro nesta página, passar para próximo livro
            pass
    return(lista_completa, lista_livros)

In [35]:
reviews, livros = raspar_skoob(1,20,35,True)

Livro 1: Ensaio Sobre a Cegueira
Livro 1: Ensaio Sobre a Cegueira - Página: 1
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 1
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 2
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 3
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 4
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 5
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 6
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 7
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 8
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 9
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 10
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 11
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 12
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 13
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 14
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 15
Livro 1: Ensaio Sobre a Cegueira - Página: 2
Livro 1: Ensaio 

Livro 5: Operação Cavalo de Tróia 1 - Página: 2
Livro 5: Operação Cavalo de Tróia 1 - Página: 2 - Review: 1
Livro 5: Operação Cavalo de Tróia 1 - Página: 2 - Review: 2
Livro 5: Operação Cavalo de Tróia 1 - Página: 2 - Review: 3
Livro 5: Operação Cavalo de Tróia 1 - Página: 2 - Review: 4
Livro 5: Operação Cavalo de Tróia 1 - Página: 2 - Review: 5
Livro 5: Operação Cavalo de Tróia 1 - Página: 2 - Review: 6
Livro 5: Operação Cavalo de Tróia 1 - Página: 2 - Review: 7
Livro 5: Operação Cavalo de Tróia 1 - Página: 2 - Review: 8
Livro 5: Operação Cavalo de Tróia 1 - Página: 2 - Review: 9
Livro 5: Operação Cavalo de Tróia 1 - Página: 2 - Review: 10
Livro 5: Operação Cavalo de Tróia 1 - Página: 2 - Review: 11
Livro 5: Operação Cavalo de Tróia 1 - Página: 2 - Review: 12
Livro 5: Operação Cavalo de Tróia 1 - Página: 2 - Review: 13
Livro 5: Operação Cavalo de Tróia 1 - Página: 2 - Review: 14
Livro 5: Operação Cavalo de Tróia 1 - Página: 2 - Review: 15
Livro 5: Operação Cavalo de Tróia 1 - Página: 

Livro 11: Pai Rico Pai Pobre - Página: 3
Livro 11: Pai Rico Pai Pobre - Página: 3 - Review: 1
Livro 11: Pai Rico Pai Pobre - Página: 3 - Review: 2
Livro 11: Pai Rico Pai Pobre - Página: 3 - Review: 3
Livro 11: Pai Rico Pai Pobre - Página: 3 - Review: 4
Livro 11: Pai Rico Pai Pobre - Página: 3 - Review: 5
Livro 12: Quem Mexeu No Meu Queijo?
Livro 12: Quem Mexeu No Meu Queijo? - Página: 1
Livro 12: Quem Mexeu No Meu Queijo? - Página: 1 - Review: 1
Livro 12: Quem Mexeu No Meu Queijo? - Página: 1 - Review: 2
Livro 12: Quem Mexeu No Meu Queijo? - Página: 1 - Review: 3
Livro 12: Quem Mexeu No Meu Queijo? - Página: 1 - Review: 4
Livro 12: Quem Mexeu No Meu Queijo? - Página: 1 - Review: 5
Livro 12: Quem Mexeu No Meu Queijo? - Página: 1 - Review: 6
Livro 12: Quem Mexeu No Meu Queijo? - Página: 1 - Review: 7
Livro 12: Quem Mexeu No Meu Queijo? - Página: 1 - Review: 8
Livro 12: Quem Mexeu No Meu Queijo? - Página: 1 - Review: 9
Livro 12: Quem Mexeu No Meu Queijo? - Página: 1 - Review: 10
Livro 12:

Livro 15: Perdas & Ganhos - Página: 3
Livro 15: Perdas & Ganhos - Página: 3 - Review: 1
Livro 15: Perdas & Ganhos - Página: 3 - Review: 2
Livro 15: Perdas & Ganhos - Página: 3 - Review: 3
Livro 15: Perdas & Ganhos - Página: 3 - Review: 4
Livro 15: Perdas & Ganhos - Página: 3 - Review: 5
Livro 15: Perdas & Ganhos - Página: 3 - Review: 6
Livro 16: Pensar é Transgredir
Livro 16: Pensar é Transgredir - Página: 1
Livro 16: Pensar é Transgredir - Página: 1 - Review: 1
Livro 16: Pensar é Transgredir - Página: 1 - Review: 2
Livro 16: Pensar é Transgredir - Página: 1 - Review: 3
Livro 16: Pensar é Transgredir - Página: 1 - Review: 4
Livro 16: Pensar é Transgredir - Página: 1 - Review: 5
Livro 16: Pensar é Transgredir - Página: 1 - Review: 6
Livro 16: Pensar é Transgredir - Página: 1 - Review: 7
Livro 16: Pensar é Transgredir - Página: 1 - Review: 8
Livro 16: Pensar é Transgredir - Página: 1 - Review: 9
Livro 16: Pensar é Transgredir - Página: 1 - Review: 10
Livro 16: Pensar é Transgredir - Pági

In [36]:
df_reviews = pd.DataFrame(reviews)
df_livros = pd.DataFrame(livros)

In [39]:
df_reviews.value_counts('book')

book
A Menina que Roubava Livros                35
Memória de minhas putas tristes            35
Perdas & Ganhos                            35
Pensar é Transgredir                       35
Pai Rico Pai Pobre                         35
Operação Cavalo de Tróia 1                 35
O Monge e O Executivo                      35
O Caçador de Pipas                         35
O Alquimista                               35
Marley & Eu                                35
Ilusões                                    35
Fernão Capelo Gaivota                      35
Ensaio Sobre a Cegueira                    35
Quem Mexeu No Meu Queijo?                  35
Investimentos                              20
O rio do meio                               4
Atitude                                     4
O amor é a melhor estratégia                3
A primeira impressão é a que fica           2
Atitude! 2 - O que você está esperando?     1
dtype: int64